# N-Puzzle Problem

In [27]:
import numpy as np
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
from icecream import ic

In [28]:
PUZZLE_DIM = 3
action = namedtuple('Action', ['pos1', 'pos2'])

In [29]:
def available_actions(state: np.array) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x,y), (x-1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x,y), (x+1, y)))
    if y > 0:
        actions.append(action((x,y), (x, y-1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x,y), (x, y+1)))
    return actions
    
    
def move(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state


In [30]:
def checkIfSolved(finalState: np.ndarray) -> bool:
    return np.array_equal(finalState, np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape(PUZZLE_DIM, PUZZLE_DIM))
    

In [44]:
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape(PUZZLE_DIM, PUZZLE_DIM)
goal_state = state.copy()
ic(checkIfSolved(state))
for r in tqdm(range(RANDOMIZE_STEPS), desc = 'Randomizing'):
    state = move(state, choice(available_actions(state)))
initial_state = state.copy()

print('Initial State:', initial_state)
print('Goal State:', goal_state)

ic| checkIfSolved(state): True
Randomizing: 100%|██████████| 100000/100000 [00:00<00:00, 110443.82it/s]

Initial State: [[2 3 4]
 [6 0 1]
 [7 5 8]]
Goal State: [[1 2 3]
 [4 5 6]
 [7 8 0]]


## Using different path search algorithms to solve the n^2-1 puzzle problem!

In [69]:
def print_solution_path_with_metrics(initial_state, solution_path):
    state = initial_state.copy()
    print("Initial State:")
    print(state)
    print("\nSteps to solve:")

    for i, action in enumerate(solution_path, start=1):
        print(f"\nStep {i}: Move tile from {action.pos2} to {action.pos1}")
        state = move(state, action)
        print(state)

    print("\nGoal State Reached!")


### BFS Algorithm

In [74]:
from collections import deque

def bfs(initial_state):
    frontier = deque([(initial_state, [])])  # here we initialize the frontier with the initial state [] and empty path
    visited = set()  # We save the visited states here as a set because it is faster to check if a state is in a set than in a list
    
    # Here we start the BFS algorithm and with steps we count the number of iterations (or levels in the tree)
    steps = 0

    while frontier: # while the frontier is not empty (we still have states to explore)
        current_state, path = frontier.popleft()  # Dequeue the next state to explore
        steps += 1 # Increment step counter for each explored state. 

        # Here we check if the current state is the goal state
        if checkIfSolved(current_state):
            quality = len(path)
            print(f"Solution Quality: {quality} actions")
            print(f"Total Cost: {steps} states evaluated")
            print(f"Efficiency: {quality / steps:.4f}")
            return path  # Return the sequence of actions to reach the goal

        # Add the current state to the visited set
        visited.add(current_state.tobytes()) # We use tobytes() to convert the numpy array to a hashable object 

        # Generate all possible actions (moves) from the current state
        for action in available_actions(current_state):
            next_state = move(current_state, action) # apply the action to the current state to get the next state

            # Only enqueue states we haven't visited
            if next_state.tobytes() not in visited:
                frontier.append((next_state, path + [action])) # Enqueue the next state and the path to reach it

    print("No solution found.")
    return None


In [75]:
solution_path = bfs(initial_state)
if solution_path:
    print_solution_path_with_metrics(initial_state, solution_path)


Solution Quality: 16 actions
Total Cost: 12670 states evaluated
Efficiency: 0.0013
Initial State:
[[2 3 4]
 [6 0 1]
 [7 5 8]]

Steps to solve:

Step 1: Move tile from (0, 1) to (1, 1)
[[2 0 4]
 [6 3 1]
 [7 5 8]]

Step 2: Move tile from (0, 2) to (0, 1)
[[2 4 0]
 [6 3 1]
 [7 5 8]]

Step 3: Move tile from (1, 2) to (0, 2)
[[2 4 1]
 [6 3 0]
 [7 5 8]]

Step 4: Move tile from (1, 1) to (1, 2)
[[2 4 1]
 [6 0 3]
 [7 5 8]]

Step 5: Move tile from (1, 0) to (1, 1)
[[2 4 1]
 [0 6 3]
 [7 5 8]]

Step 6: Move tile from (0, 0) to (1, 0)
[[0 4 1]
 [2 6 3]
 [7 5 8]]

Step 7: Move tile from (0, 1) to (0, 0)
[[4 0 1]
 [2 6 3]
 [7 5 8]]

Step 8: Move tile from (0, 2) to (0, 1)
[[4 1 0]
 [2 6 3]
 [7 5 8]]

Step 9: Move tile from (1, 2) to (0, 2)
[[4 1 3]
 [2 6 0]
 [7 5 8]]

Step 10: Move tile from (1, 1) to (1, 2)
[[4 1 3]
 [2 0 6]
 [7 5 8]]

Step 11: Move tile from (1, 0) to (1, 1)
[[4 1 3]
 [0 2 6]
 [7 5 8]]

Step 12: Move tile from (0, 0) to (1, 0)
[[0 1 3]
 [4 2 6]
 [7 5 8]]

Step 13: Move tile from (

In [76]:
# Call the BFS function
solution_path = bfs(initial_state)

# Check if a solution was found
if solution_path is not None:
    print("Solution path:", solution_path)
else:
    print("No solution found.")

Solution Quality: 16 actions
Total Cost: 12670 states evaluated
Efficiency: 0.0013
Solution path: [Action(pos1=(1, 1), pos2=(0, 1)), Action(pos1=(0, 1), pos2=(0, 2)), Action(pos1=(0, 2), pos2=(1, 2)), Action(pos1=(1, 2), pos2=(1, 1)), Action(pos1=(1, 1), pos2=(1, 0)), Action(pos1=(1, 0), pos2=(0, 0)), Action(pos1=(0, 0), pos2=(0, 1)), Action(pos1=(0, 1), pos2=(0, 2)), Action(pos1=(0, 2), pos2=(1, 2)), Action(pos1=(1, 2), pos2=(1, 1)), Action(pos1=(1, 1), pos2=(1, 0)), Action(pos1=(1, 0), pos2=(0, 0)), Action(pos1=(0, 0), pos2=(0, 1)), Action(pos1=(0, 1), pos2=(1, 1)), Action(pos1=(1, 1), pos2=(2, 1)), Action(pos1=(2, 1), pos2=(2, 2))]


In [37]:
state

array([[0, 4, 8],
       [2, 6, 7],
       [3, 5, 1]])

### DFS Algorithm

In [ ]:
def dfs(initial_state, max_depth=60):
    stack = [(initial_state, [])]  # (current state, path of actions leading to state)
    visited = set()  # To track visited states
    
    # Track number of steps to reach the solution
    steps = 0

    while stack:
        current_state, path = stack.pop()  # Pop the last state from the stack
        steps += 1

        # Check if the current state matches the goal state
        if checkIfSolved(current_state):
            quality = len(path)
            print(f"Solution Quality: {quality} actions")
            print(f"Total Cost: {steps} states evaluated")
            print(f"Efficiency: {quality / steps:.4f}")
            return path

        # Add the current state to the visited set
        visited.add(current_state.tobytes())

        # Generate all possible actions (moves) from the current state
        for action in available_actions(current_state):
            next_state = move(current_state, action)

            # Only push states we haven't visited and are within depth limits
            if next_state.tobytes() not in visited and len(path) < max_depth:
                stack.append((next_state, path + [action]))

    print("No solution found.")
    return None


In [95]:

dfs_solution_path = dfs(initial_state, max_depth=60)
if dfs_solution_path:
    print_solution_path_with_metrics(initial_state, dfs_solution_path)
print(dfs_solution_path)

Solution Quality: 58 actions
Total Cost: 24141 states evaluated
Efficiency: 0.0024
Initial State:
[[2 3 4]
 [6 0 1]
 [7 5 8]]

Steps to solve:

Step 1: Move tile from (1, 2) to (1, 1)
[[2 3 4]
 [6 1 0]
 [7 5 8]]

Step 2: Move tile from (2, 2) to (1, 2)
[[2 3 4]
 [6 1 8]
 [7 5 0]]

Step 3: Move tile from (2, 1) to (2, 2)
[[2 3 4]
 [6 1 8]
 [7 0 5]]

Step 4: Move tile from (2, 0) to (2, 1)
[[2 3 4]
 [6 1 8]
 [0 7 5]]

Step 5: Move tile from (1, 0) to (2, 0)
[[2 3 4]
 [0 1 8]
 [6 7 5]]

Step 6: Move tile from (1, 1) to (1, 0)
[[2 3 4]
 [1 0 8]
 [6 7 5]]

Step 7: Move tile from (1, 2) to (1, 1)
[[2 3 4]
 [1 8 0]
 [6 7 5]]

Step 8: Move tile from (2, 2) to (1, 2)
[[2 3 4]
 [1 8 5]
 [6 7 0]]

Step 9: Move tile from (2, 1) to (2, 2)
[[2 3 4]
 [1 8 5]
 [6 0 7]]

Step 10: Move tile from (2, 0) to (2, 1)
[[2 3 4]
 [1 8 5]
 [0 6 7]]

Step 11: Move tile from (1, 0) to (2, 0)
[[2 3 4]
 [0 8 5]
 [1 6 7]]

Step 12: Move tile from (1, 1) to (1, 0)
[[2 3 4]
 [8 0 5]
 [1 6 7]]

Step 13: Move tile from (

### A* Algorithm